In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tenten_v2 import makematrix
from tenten_v2 import checkmatrix


In [ ]:
def mixer(CV,streams):
  """
  generic mixer function takes in a CV that is a mixer
  input streams and single output streams are defined in the CV variable
  streams is all our stream component flows in the whole system

  we apply a simple mass balance, but to the correct streams
  """

  in_index = 1
  out_index = 2
  strs_in = CV[in_index]     # numbers of the inlet streams
  str_out = CV[out_index]    # numbers of the outlet streams

  #balance equation accesses the particular stream values of interest
  #ins = outs
  acc = (streams[strs_in[0]] + streams[strs_in[1]])  -  (streams[str_out[0]])

  return acc

In [ ]:
def mixertree(X):

  """blending problem
  7 streams, 2 components, 3 mixers
h ere, the stream connectivity
 0 + 1 = 2 in mixer 1
2 + 3 = 4 in mixer 2
5+ 4 = 6 in mixer 3
  """

  num_streams = 7
  num_compts = 2
  streams = X.reshape( (num_streams,num_compts) )

  CV =  [   [["mixer"], [0,1], [2]],  \
            [["mixer"] ,[2,3], [4]],  \
            [["mixer"], [5,4], [6]]     ]

  # initialize n and eq
  n = 0
  eq = np.zeros(X.size)


  # loop over CVs
  for i in CV:
      if i[0][0] == "mixer":
        eq[n:n+num_compts] = mixer(i,streams)
      n = n+num_compts



  # add final constraints
  eq[n]   = streams[0,0]  - 100    # stream 0 pure A
  eq[n+1] = streams[0,1]  - 0    # stream 0

  eq[n+2] = streams[1,0]  - 0    # stream 1 pure B
  eq[n+3] = streams[1,1]  - 150    # stream 1

  eq[n+4] = streams[3,0] - 75    # stream 3 75/25
  eq[n+5] = streams[3,1] - 25    # stream 3

  eq[n+6] = streams[5,0] - 5    # stream 5 lots of B
  eq[n+7] = streams[5,1] - 95   # stream 5


  return eq



In [ ]:
A,b = makematrix(mixertree,14)
checkmatrix(A)
solution = np.linalg.solve(A, b)

print(solution)
streams = solution.reshape( (7,2) )
print(f"\n\n{streams}")

# of equations m = 14 and # of unknowns N = 14
[100.   0.  -0. 150. 100. 150.  75.  25. 175. 175.   5.  95. 180. 270.]


[[100.   0.]
 [ -0. 150.]
 [100. 150.]
 [ 75.  25.]
 [175. 175.]
 [  5.  95.]
 [180. 270.]]
